# Introduction to Data Science – Homework 6
*COMP 5360 / MATH 4100, University of Utah, http://datasciencecourse.net/*

Due: Friday, March 03 2023, 11:59pm.

In Part 1 of this homework you will scrape github repositories and organize the information in a Pandas dataframe. In Part 2, you will use linear regression to gain meaningful insights. 

## Your Data
First Name: Melissa
<br>
Last Name: Frisby
<br>
E-mail: u1175875@utah.edu
<br>
UID: u1175875
<br>

In [1]:
# imports and setup 
from bs4 import BeautifulSoup
# you can use either of these libraries to get html from a website
import time
import os

import pandas as pd
import scipy as sc
import numpy as np

import statsmodels.formula.api as sm

import matplotlib.pyplot as plt 
plt.style.use('ggplot')
%matplotlib inline  
plt.rcParams['figure.figsize'] = (10, 6) 
# where the data is stored
DATA_PATH = "data"

### 1. Scrape Github Repository List using BeautifulSoup
In this part you will explore Github repositories, specifically the 100 most-starred repositories. You are going to scrape data from [this repository list](https://github.com/search?o=desc&q=stars%3A%3E1&s=stars&type=Repositories).

### 1.1. Check whether you are permitted to scrape the data
Before you start to scrape any website you should go through the terms of service and policy documents of the website. Almost all websites post conditions to use their data. Check the terms of [https://github.com/](https://github.com/)(see the tiny "terms" link at the bottom of the page) to see whether the site permits you to scrape their data or not. Are you sure you are allowed to scrape?

**Your solution:**
Yes, but with some limitations of how often to prevent getting the account blocked.

### Task 1.2 Load the Data

To avoid any problems with GitHub blocking us from downloading the data many times, we have downloaded and saved all the html files in the [data](data) folder. Note that the data folder is not completely consistent with what you see on the web – we've made a few patches to the data that makes your task here easier. You will be treating the data folder as your website to be scraped. The path to data folder is stored in `DATA_PATH` variable.

In the data folder you will find first 10 pages of highly starred repositories saved as `searchPage1.html`,`searchPage2.html`,`searchPage3.html` ... `searchPage10.html`

Check out page 10 if you want to see what happens if you scrape to quickly ;). 

Now read these html files in python and create a soup object. This is a two step process:
 * Read the text in the html files
 * Create the soup from the files that you've read. 

In [2]:
## Your code goes here
file = [0]*10

for x in range(len(file)):
    file[x] = BeautifulSoup(open(f'data/searchPage{x+1}.html', errors = 'ignore'), 'html.parser')
    
print(file[1].prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <link href="https://github.githubassets.com" rel="dns-prefetch"/>
  <link href="https://avatars0.githubusercontent.com" rel="dns-prefetch"/>
  <link href="https://avatars1.githubusercontent.com" rel="dns-prefetch"/>
  <link href="https://avatars2.githubusercontent.com" rel="dns-prefetch"/>
  <link href="https://avatars3.githubusercontent.com" rel="dns-prefetch"/>
  <link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
  <link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
  <link crossorigin="anonymous" href="https://github.githubassets.com/assets/frameworks-6548d5a1dd84bd83036c6a914b25b4ae.css" integrity="sha512-ZUjVod2EvYMDbGqRSyW0rpfgBq3i+gnR/4PfrzLsy5f20oIcRfgFQFVKgi3Ztp917bP1K/kdP5q8+nAlJ3+cFA==" media="all" rel="stylesheet"/>
  <link crossorigin="anonymous" href="https://github.githubassets.com/assets/site-1cd57b56e5c8f7c49216299a13bafc79.css" integrity="sha512-HNV7VuXI9

### Extracting Data

Extract the following data for each repository, and create a Pandas Dataframe with a row for each repository and a column for each of these datums. 

+ The name of the repository
+ The primary language (there are multiple or none, if multiple, use the first one, if none, use "none")
+ The number of watches
+ The number of stars
+ Number of commits
+ Number of branches
+ Number of contributors
+ Length of readme file.
+ Number of pull requests

Here's an example for one repository on how that data could look like: 
```python
{'name': 'Python-100-Days',
'language': 'Jupyter Notebook',
'watches': '4822',
'stars': '78068',
'commits': 296,
'branches': 1,
'contributors': 12,
'readme': 11159,
'pull_request':85}
```

### Task 1.3 Extract repository URLs

If you look at the results of the 100 most-starred repositories [(this list)](https://github.com/search?o=desc&q=stars%3A%3E1&s=stars&type=Repositories), you will notice that all the information we want to extract for each repository is not in that list. This information is in the repository’s individual web page, for example [996icu](https://github.com/996icu/996.ICU). 

Therefore, you will first have to extract links of each repository from the soup you scraped earlier. When you extract the link for the repository, it will be a path to the stored HTML page for the repository. You will use this path to read the file and extract the above information.

Refer to the scraping lecture for details on how to do this. We recommend you use the web inspector to identify the relevant structures.

Example of a link that you need to extract - 996icu/996.ICU.html

In [3]:
links = {}

for content in file:
    for a in content.select('a.v-align-middle'):
        link = a.get('href')
        text = a.get_text().split(" - ")
        links[text[0].strip()] = (link)
print(links)

{'freeCodeCamp/freeCodeCamp': 'freeCodeCamp/freeCodeCamp.html', '996icu/996.ICU': '996icu/996.ICU.html', 'vuejs/vue': 'vuejs/vue.html', 'facebook/react': 'facebook/react.html', 'tensorflow/tensorflow': 'tensorflow/tensorflow.html', 'twbs/bootstrap': 'twbs/bootstrap.html', 'EbookFoundation/free-programming-books': 'EbookFoundation/free-programming-books.html', 'sindresorhus/awesome': 'sindresorhus/awesome.html', 'getify/You-Dont-Know-JS': 'getify/You-Dont-Know-JS.html', 'ohmyzsh/ohmyzsh': 'ohmyzsh/ohmyzsh.html', 'jwasham/coding-interview-university': 'jwasham/coding-interview-university.html', 'kamranahmedse/developer-roadmap': 'kamranahmedse/developer-roadmap.html', 'github/gitignore': 'github/gitignore.html', 'airbnb/javascript': 'airbnb/javascript.html', 'microsoft/vscode': 'microsoft/vscode.html', 'CyC2018/CS-Notes': 'CyC2018/CS-Notes.html', 'd3/d3': 'd3/d3.html', 'flutter/flutter': 'flutter/flutter.html', 'torvalds/linux': 'torvalds/linux.html', 'facebook/react-native': 'facebook/r

### Task 1.4 Extracting required information

Once you have extracted links for each repository, you can start parsing those HTML pages using BeautifulSoup and extract all the required information.

**Note: There are few repositories which do not contain 'issues' field (such as 996icu/996.ICU.html). Therefore, write your code such that it handles this condition as well.**

Save the dataframe you created to a new file project_info.csv and *include this in your submission*. You will loose points if you don't.

You also need to make sure that you reformat all numerical columns to be integer data. You can do that either as you parse, or when you have a dataframe with strings.

Note that there is one repository flagged as having infinite contributers (the Linux kernel). We'll assume that it in fact has 15600 contributors (an estimate based on a google search).

In [9]:
import urllib.request
url = 'https://github.com/996icu/996.ICU'

with urllib.request.urlopen(url) as response:
    html = response.read()
    html = html.decode('utf-8')
    

# need the Stars, Commits, Pull Requests, Branches, Watchers, Contributors
link_996icu = BeautifulSoup(html, 'html.parser')
link_996icu

#project_info.to_csv('project_info.csv', index=False)


<!DOCTYPE html>

<html data-a11y-animated-images="system" data-color-mode="auto" data-dark-theme="dark" data-light-theme="light" lang="en">
<head>
<meta charset="utf-8"/>
<link href="https://github.githubassets.com" rel="dns-prefetch"/>
<link href="https://avatars.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
<link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
<link crossorigin="" href="https://github.githubassets.com" rel="preconnect"/>
<link href="https://avatars.githubusercontent.com" rel="preconnect"/>
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/light-fe3f886b577a.css" media="all" rel="stylesheet"><link crossorigin="anonymous" href="https://github.githubassets.com/assets/dark-a1dbeda2886c.css" media="all" rel="stylesheet"><link crossorigin="anonymous" data-color-theme="dark_dimmed" data-href="https://github.githubassets.com/assets/dark_dimmed-1ad5cf51dfeb.c

### 2. Analyzing the repository data

In this part, you will analyze the data collected in Part 1 using regression tools. The goal is to identify properties that make a repository popular. 

First, load the `project_info.csv` file in again. **We need you to do this so that we can run your code below without having to run your scraping code, which can be slow.**

In [ ]:
project_info = pd.read_csv('project_info.csv')
project_info.head()

### Task 2.1 Describe the data

+ Get an overview of the data using the describe function.
+ Compute the correlation matrix, visualize it with a heat map.
+ Visualize the correlations by making a scatterplot matrix.
+ Interprete what you see.

You can re-use code from your previous homework here.

In [ ]:
# your code goes here

**Your Interpretation:** TODO

### 2.2. Linear regression

1. Use linear regression to try to predict the number of Stars based on Commits, Pull Requests, Branches. Explain why this is not a very good model by discussing the R-squared , F-statistic p-value, and coefficient  p-values. 
+ Develop another model which is better. Explain why it is better and interpret your results. Hint: try using variable Watches and/or Contributors. 

In [ ]:
# your code goes here

**Your interpretation:** TODO